Link to Pack 1 — https://www.kaggle.com/code/tonylica/community-baselines-my-arc-10-pack-1-10/

Link to Pack 2 — https://www.kaggle.com/code/tonylica/community-baselines-my-arc-10-pack-1-10/

Shout-out to @kenkrige for the awesome, gold-winning, and super clear notebooks:

(1) Flip & Zip — https://www.kaggle.com/code/kenkrige/chipping-skills-1-flip-and-zip

(2) Pattern Copy — https://www.kaggle.com/code/kenkrige/chipping-skills-2-pattern-copy

(3) Regex — https://www.kaggle.com/code/kenkrige/chipping-skills-3-regex

Notebook code now from Garry Moss : https://www.kaggle.com/code/garrymoss/golfed-solutions-and-explanations-to-hard-problems
I’d like to share some of what I’m currently working on. It’s not perfect, but it may still be the strongest public baseline so far. I’m especially curious how those top scores were achieved.

I like to share a total of 100 solutions, posting 10 at a time.

In [ ]:
!pip install zopfli

import os
import zipfile
import re
import zopfli.zlib
from rich import print as print_rich
from rich.syntax import Syntax
from rich.panel import Panel

# --- Official Utilities Setup ---
# Ensures the helper functions for loading and showing examples are available.
import sys
sys.path.append("/kaggle/input/google-code-golf-2025/code_golf_utils")
from code_golf_utils import *

# --- Configuration ---
source = "/kaggle/input/google-code-golf-2025-submit"
SUBMISSION_DIR = "/kaggle/working/submission"
submission = "/kaggle/working/submission"
os.makedirs(SUBMISSION_DIR, exist_ok=True)
os.chdir(submission)


# Copy tasks into submission folder
processed_tasks = 0
for task_num in range(1, 401):
    path_in = f"{source}/task{task_num:03d}.py"
    path_out = f"{submission}/task{task_num:03d}.py"
    
    if not os.path.exists(path_in):
        continue
    
    try:
        with open(path_in, "rb") as fin:
            code = fin.read()
        with open(path_out, "wb") as fout:
            fout.write(code)
        processed_tasks += 1
    except Exception as e:
        print(f"Error processing task{task_num:03d}: {e}")

print(f"Processed {processed_tasks} tasks")


def save_final_solution(task_id: int, solution_code: str):
    """
    Analyzes the solution code, compresses it with zopfli if beneficial, 
    and saves the final version to the submission directory.
    """
    file_path = os.path.join(SUBMISSION_DIR, f"task{task_id:03d}.py")
    
    solution_bytes = solution_code.strip().encode('utf-8')
    original_len = len(solution_bytes)
    
    # --- Compression Logic ---
    compressed_data = zopfli.zlib.compress(solution_bytes)
    compressed_data = b"'" + compressed_data + b"'" if b'"' in compressed_data else b'"' + compressed_data + b'"'
    compressed_code_str = b"#coding:L1\nimport zlib;exec(zlib.decompress(bytes(%s,'L1')))" % compressed_data
    compressed_len = len(compressed_code_str)

    final_bytes_to_save = solution_bytes
    used_compression = False
    
    if compressed_len < original_len:
        final_bytes_to_save = compressed_code_str
        used_compression = True
        print_rich(f"[bold green]Compression successful![/bold green] Original: {original_len} bytes -> Final: {compressed_len} bytes")
    else:
        print_rich(f"[bold yellow]No compression improvement.[/bold yellow] Original size: {original_len} bytes")
    
    with open(file_path, 'wb') as f:
        f.write(final_bytes_to_save)

    # Display what was saved for verification
    display_code = final_bytes_to_save.decode('utf-8', errors='replace') if used_compression else solution_code
    panel_title = f"[bold]✓ Saved solution for Task {task_id}[/bold] ({len(final_bytes_to_save)} bytes)"
    panel = Panel(
        Syntax(display_code.strip(), "python", theme="monakai", line_numbers=False),
        title=panel_title, border_style="green"
    )
    print_rich(panel)


def create_submission_zip():
    """
    Finds all saved task files and packages them into submission.zip.
    """
    zip_path = "/kaggle/working/submission.zip"
    print_rich(f"\nCreating submission file at [cyan]{zip_path}[/cyan]...")
    
    saved_solutions = [
        f for f in os.listdir(SUBMISSION_DIR) 
        if f.startswith('task') and f.endswith('.py')
    ]

    with zipfile.ZipFile(zip_path, 'w', zipfile.ZIP_DEFLATED) as zipf:
        for filename in sorted(saved_solutions):
            file_path = os.path.join(SUBMISSION_DIR, filename)
            zipf.write(file_path, arcname=filename)
    
    solved_count = len(saved_solutions)
    print_rich(f"[bold green]✓ Done![/bold green]")
    print_rich(f"Packaged [yellow]{solved_count}[/yellow] solutions into the submission file.")
    print_rich("You can now submit '/kaggle/working/submission.zip'.")


# Display the color legend for the grids
show_legend()

# How I Golfed ARC **Task 178 – “layercake”** (and why <70B is so hard)

**Task:** compress a grid by

1. dropping **adjacent duplicate rows**, and
2. collapsing **adjacent equal cells** within each kept row.

**Goal:** tiny, **Py3-safe** one-liners that pass all tests.

Below are **four** working styles I actually used, what they do, why they differ in bytes, and the core tricks behind them.

---

## 1) Two-phase “row then dedupe rows”

```python
p=lambda g:(h:=[[b for a,b in zip([0]+r,r)if a^b]for r in g])and[r for a,r in zip([[]]+h,h)if a!=r]
```

**Idea:** first shrink **each row** (`h`), then remove **consecutive duplicate rows** from `h`.
**Pros:** explicit, easy to reason about.
**Cons:** the temp `h` + the second `zip` costs bytes.

---

## 2) Single-pass list-comp (classic)

```python
p=lambda g:[[b for a,b in zip([0]+a,a)if a^b]for a,b in zip(g,[[]]+g)if a!=b]
```

**Idea:** inline the **row compression** and the **adjacent-row filter** in one pass.
**Pros:** tight and pure Py3; rows are **lists**.
**Cons:** still pays for **two zips** (inner per-row and outer across rows).

---

## 3) “Call-It-Twice” helper (axis-agnostic)

```python
p=lambda g,f=lambda d:(x:=0)or[*zip(*[(x:=b)for b in d if x!=b])]:f(f(g))
```

**Idea:** define `f` as **“dedupe along rows, then transpose”**. Calling `f` **twice** hits rows, then columns, and restores orientation.
**Pros:** beautifully symmetric; avoids separate row/col code; short.
**Cons:** rows become **tuples** (often fine). Still lands in the **low-70s** in Py3.

---

## 4) Recursion template (2 ticks = rows then cols)

```python
p=lambda d,t=1:-t*d or p([*zip(*(b for a,b in zip([0]+d,d)if a!=b))],t-1)
```

**Idea:** run the **same step twice**: “dedupe-then-transpose” at `t=1`, then again at `t=0`.
**Pros:** clean, reusable template; Py3-safe; outputs **list of tuples**.
**Cons:** the list-materialization `[*zip(...)]` is the byte wall.

---

## Why **<70B** is hard

* In **Python 3**, `zip` is an **iterator**. To (a) satisfy list/tuple outputs and (b) keep the recursion guard simple, we must **materialize**: `[*zip(...)]`. That star-splat is the stubborn ~3-byte tax.
* The task inherently needs **two scans**:

  * inner **row dedupe** (`zip([0]+row, row)` or a walrus sentinel), and
  * outer **adjacent-row** filter (`zip(g, [prev]+g)` or a sentinel).
* `a^b` is byte-cheaper than `a!=b` **inside rows** (numbers), but you can’t XOR lists/tuples for the outer row-vs-row check—so you still pay for `!=` somewhere.

---

## The core trick: **Call-It-Twice**

Define a tiny helper that always does:

1. **row-wise transform**, then
2. **transpose**.

Calling it **twice**:

* first time: solves the row axis, rotates,
* second time: solves the column axis (now rows), rotates back.

That’s what #3 and #4 encode compactly.

---

## Byte tips 

* Prefer `a^b` over `a!=b` **inside** a numeric row.
* `zip([0]+d,d)` is usually shorter than maintaining a “previous” row with walrus.
* If rows can be **tuples**, keep them—`zip(*...)` already yields tuples for free.
* For the “same step twice” pattern, the **recursion template** is a great shell:

  ```python
  p=lambda d,t=1:-t*d or p(STEP(d),t-1)
  ```

---

## Which one to use?

* Want **plain lists** and one pass? → **#2**.
* Want **axis-agnostic** code that’s easy to reuse? → **#3** (helper) or **#4** (recursion).
* Prefer explicit staging for clarity/testing? → **#1**.

If you manage to shave those last few bytes, I’m all ears—drop your trick!


In [ ]:
TASK_ID = 178
current_task_data = load_examples(TASK_ID)
show_examples(current_task_data['train'] + current_task_data['test'])

p=lambda g,f=lambda d:(x:=0)or[*zip(*[(x:=b)for b in d if x!=b])]:f(f(g))

cell_source = _ih[-1]
last_solution = re.findall(r"(def p\(.*?\):.*?|p=lambda.*?:.*?)(?=\n\n|\n# ---|\n$)", cell_source, re.S)[-1]
save_final_solution(TASK_ID, last_solution)

## Task 327 explaintion
@Vitaly Gatsko 

Task 327 can be written as a lambda and brought down to ~101 bytes:

R=range(6);T=0,1,2;p=lambda g:[[sum((i-r==j-c>=0)*g[r][c]for r in T for c in T)for j in R]for i in R]

A small indexing trick shortens it to ~98 bytes:

R=range;p=lambda g:[[sum((i-r%3==j-r//3>=0)*g[r%3][r//3]for r in R(9))for j in R(6)]for i in R(6)]

@JoKing  Hint: compute each output row from the previous one. “Take the previous row, prepend a 0 (shift right), then overlay the current seeds.” Sketch:

p=lambda g,l=...:[l:=f(i,[0]+l)for i in ...]

@Tony Li   Exactly—escaping the ~100B range hinges on that DP idea:

My (rough) ~72B attempt (typos fixed below).

What the two styles are doing

Diagonal-sum style (101 → 98B) You treat the 3×3 input g as “seeds” and, for each output cell (i,j) in 6×6, sum the seeds that align on the same down-right diagonal:

i-r%3 == j-r//3 >= 0 checks alignment and bounds.

The 98B version compresses the double loop over (r,c) into a single loop r in range(9) with r%3, r//3 to index g.

DP row-builder (the hint) Keep the previous output row l. For each input row r:

shift: prepend 0 to l and trim to width 6 → (0,*l)[:6]

overlay: take elementwise max with r padded to width 6 → r+[0]*6

The new l becomes the next output row. Pad with three empty rows to “flush” the shift all the way down to produce exactly 6 rows.

In [ ]:
TASK_ID = 327
current_task_data = load_examples(TASK_ID)
show_examples(current_task_data['train'] + current_task_data['test'])

p=lambda g,l=[0]*6:[(l:=[*map(max,[0]+l,r+[0]*6)][:6])for r in g+3*[[]]]

cell_source = _ih[-1]
last_solution = re.findall(r"(def p\(.*?\):.*?|p=lambda.*?:.*?)(?=\n\n|\n# ---|\n$)", cell_source, re.S)[-1]
save_final_solution(TASK_ID, last_solution)

## task359 Solution authored by: henrychibueze (aka "Ches Charlemagne")¶

Full credit to the original author—shared here with thanks.

Idea: For each cell, combine its row and column into a single list and choose the color that appears most frequently (the mode). This suppresses noise pixels while preserving the underlying stripe colors.

Step-by-step explanation:

for r in d: iterate over each row r.

zip(d): transpose the grid to get all columns as tuples; with a fixed outer row, the inner “for C in zip(d)” pairs that row with every column C, i.e., “make a decision for each (row, column) intersection.”

S := r + [C]: build a list S by concatenating the current row r and the current column C. Here, [C] converts the column tuple to a list, and := is the walrus operator so we can reuse S immediately. This acts as a multiset union of row ∪ column (multiplicities preserved).

max(S, key=S.count): pick the most frequent color (the mode) within S. Because stripes are constant across entire rows or columns and the noise is sparse, the stripe color overwhelmingly dominates S and gets selected.

The double list comprehension assembles that per-intersection choice into the full output grid.

Why this works for both horizontal and vertical stripes:

Horizontal stripes: the row’s stripe color already dominates S, so the row color wins. Vertical stripes: likewise, the column’s stripe color dominates S, so the column color wins. No extra “detect stripe direction” branch is needed—short and robust. Notes (edge cases):

If a rare tie occurs (two colors with equal counts), Python’s max keeps the first occurrence. Since we concatenate row r before column C, ties are biased toward the row’s color. In the generator, the stripe’s advantage is large, so this practically never matters.

@bizy-coder :Getting this down to the gold of 64 bytes is pretty easy.

You can unpack C during the comprehension, to avoid needing [*C].

p=lambda d:[[max(S:=r+C,key=S.count)forC,in zip(d)]for r in d]

@JoKing Hell, you can do straight up list assignments in comprehension loops like [1for g[a][b]in[1]] or [...for g[::-1]in g]

In [ ]:
TASK_ID = 359
current_task_data = load_examples(TASK_ID)
show_examples(current_task_data['train'] + current_task_data['test'])

p=lambda d:[[max(S:=r+C,key=S.count)for*C,in zip(*d)]for r in d]

cell_source = _ih[-1]
last_solution = re.findall(r"(def p\(.*?\):.*?|p=lambda.*?:.*?)(?=\n\n|\n# ---|\n$)", cell_source, re.S)[-1]
save_final_solution(TASK_ID, last_solution)

@JoKing

p=lambda g:[(g:=[*zip(*((len(g)-len(u:=[r for r in g if r.count(max(next(filter(any,g))))!=1]))*g[:1]+u))][::-1])for _ in[0]*4][3]
=> remove extra brackets => 126b
p=lambda g:[g:=[*zip(*(len(g)-len(u:=[r for r in g if r.count(max(next(filter(any,g))))!=1]))*g[:1]+u)][::-1]for _ in[0]*4][3]
=> switch out [0]*4 with a pre-existing list since we don't care about its contents (and barely care about length) => 123b
p=lambda g:[g:=[*zip(*(len(g)-len(u:=[r for r in g if r.count(max(next(filter(any,g))))!=1]))*g[:1]+u)][::-1]for _ in g][3]
=> truncate after rather than work out an exact prefix => 119b
p=lambda g:[g:=[*zip(*(g[:1]*11+[r for r in g if r.count(max(next(filter(any,g))))!=1])[-len(g):])][::-1]for _ in g][3]
=> redacted => -3 bytes
=> redacted => -5 bytes

Erm, I suppose i may have golfed it a bit too much and ended up past the scores on the public scoresheet at 111 bytes. The next couple steps aren't particularly difficult (and are independent of each other), so I'll leave them as an exercise to the reader.

In [ ]:
TASK_ID = 234
current_task_data = load_examples(TASK_ID)
show_examples(current_task_data['train'] + current_task_data['test'])

p=lambda g:[g:=[*zip(*(g[:1]*11+[r for r in g if r.count(max(next(filter(any,g))))!=1])[-len(g):])][::-1]for _ in g][3]

cell_source = _ih[-1]
last_solution = re.findall(r"(def p\(.*?\):.*?|p=lambda.*?:.*?)(?=\n\n|\n# ---|\n$)", cell_source, re.S)[-1]
save_final_solution(TASK_ID, last_solution)

In [ ]:
TASK_ID = 377
current_task_data = load_examples(TASK_ID)
show_examples(current_task_data['train'] + current_task_data['test'])

def p(g):B=max((A[:1]+[A for(B,A)in zip(A,A[1:])if B^A]for A in g),key=len);A=len(B);C=range(A);return[[B[min(D,C,~D+A,~C+A)]for C in C]for D in C]

cell_source = _ih[-1]
last_solution = re.findall(r"(def p\(.*?\):.*?|p=lambda.*?:.*?)(?=\n\n|\n# ---|\n$)", cell_source, re.S)[-1]
save_final_solution(TASK_ID, last_solution)

In [ ]:
TASK_ID = 110
current_task_data = load_examples(TASK_ID)
show_examples(current_task_data['train'] + current_task_data['test'])

def p(g):R=range;f=lambda A:next(i for i in R(1,30)if 1>any(a*b*(a^b)for r in A for a,b in zip(r,r[i:])));t=f(g);s=f(zip(*g));return[[max(max(r[j%t::t])for r in g[i%s::s])for j in R(29)]for i in R(29)]

cell_source = _ih[-1]
last_solution = re.findall(r"(def p\(.*?\):.*?|p=lambda.*?:.*?)(?=\n\n|\n# ---|\n$)", cell_source, re.S)[-1]
save_final_solution(TASK_ID, last_solution)

@MassimilianoGhiotto 's TASK 154 solution

https://www.kaggle.com/code/massimilianoghiotto/problem-154-what-i-learn-so-far

In [ ]:
TASK_ID = 154
current_task_data = load_examples(TASK_ID)
show_examples(current_task_data['train'] + current_task_data['test'])

p=lambda g:[(g:=[[*map(int,__import__('re').sub(r'(.*5.*)(020)(.*)\2(.*)',lambda m:'0'*len(x:=m[1])+m[2]+x[::-1]+m[3][len(x):]+m[2]+m[4],str(r)[1::3]))]for r in zip(*g[::-1])])for _ in g*4][-1]

cell_source = _ih[-1]
last_solution = re.findall(r"(def p\(.*?\):.*?|p=lambda.*?:.*?)(?=\n\n|\n# ---|\n$)", cell_source, re.S)[-1]
save_final_solution(TASK_ID, last_solution)

In [ ]:
create_submission_zip()